In [2]:
import cv2
import numpy as np
import imutils
import time

def barcode_detector(image):
        
        #image = cv2.imread("images/Barcode.jpg")
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        gradX = cv2.Sobel(gray, ddepth = cv2.cv.CV_32F, dx = 1, dy = 0, ksize = -1)
        gradY = cv2.Sobel(gray, ddepth = cv2.cv.CV_32F, dx = 0, dy = 1, ksize = -1)
        
        
        #To find regions in image with high horizontal gradients and low vertical gradients
        gradient = cv2.subtract(gradX, gradY)
        gradient = cv2.convertScaleAbs(gradient)

        blurred  = cv2.blur(gradient , (11,11)) #Blurring to filter out noise in the image
        _,threshold  = cv2.threshold(gradient , 245, 255 , cv2.THRESH_BINARY)

        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (21, 7))
        closed = cv2.morphologyEx(threshold , cv2.MORPH_CLOSE , kernel)
        
        #To get the roi that is barcode and reducing the size of other blobs
        closed = cv2.erode(closed, None, iterations  = 5) 
        closed = cv2.dilate(closed, None, iterations = 4)
        
        #Finding contours
        cnts,_ = cv2.findContours(closed.copy(), cv2.RETR_EXTERNAL , cv2.CHAIN_APPROX_SIMPLE)
        
        if len(cnts) == 0:
            return None
        
        c = sorted(cnts , key = cv2.contourArea  , reverse = True)[0]

        rect = cv2.minAreaRect(c)
        box = np.int0(cv2.cv.BoxPoints(rect))
        

        return box
    
    #Getting the video stream from camera
cap = cv2.VideoCapture(0)
time.sleep(2)


while True:
    
    ret,frame = cap.read()
    
    box = barcode_detector(frame)
    
    cv2.drawContours(frame, [box], -1, (0, 255, 0), 3)
    
    imutils.resize(frame,360,400)
    
    cv2.imshow("Barcode",frame)
    if  cv2.waitKey(1) == 13:
        break
    
cap.release()    
cv2.destroyAllWindows()